In [ ]:
%tensorflow_version 1.x

In [ ]:
pip install numpy==1.16

For the code cell below you should see ‘Found GPU’ and tf version 1.x. Else remember to change runtime to GPU

In [ ]:
!nvidia-smi

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Clome the Tensorflow API repo

In [ ]:
!git clone https://github.com/tensorflow/models.git

Install some needed tools and dependencies.

In [ ]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'


Always run the cell below for every session restart

In [ ]:
!python setup.py build
!python setup.py install

Check thr remaining time allowed for the session you are runing. Depending on your data try to know the number of hours that will be enough. If the time is lesser then restart the session. NB the time result you see in in hours

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Run Code below to see if all we need for the training has been installed.

In [ ]:
#rember the last CD you did in order to specify the directory.
!pip install git+https://github.com/google-research/tf-slim.git
%cd /content/models/research/object_detection/builders/
!python model_builder_test.py

Change Directory to object detection folder

In [ ]:
%cd /content/models/research/object_detection/

Get the pre-trained Object detection model from TensorFlow with the code below

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -xvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

In [ ]:
pip install tensorboardcolab

Ensure to run the code cell below first

In [ ]:
%load_ext tensorboard

IF you want to keep track of your model loss it best to run tensor board in background just like this before you run your training

In [ ]:
%tensorboard --logdir training/

In [ ]:
LOG_DIR = '/tmp/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
with open ('ssd_mobilenet_v2_coco.config', 'w') as rsh:
    rsh.write('''\
    # Quantized trained SSD with Mobilenet v2 on Pascal VOC Dataset.
    model {
      ssd {
        num_classes: 3
        image_resizer {
          fixed_shape_resizer {
            height: 300
            width: 300
          }
        }
        feature_extractor {
          type: "ssd_mobilenet_v2"
          depth_multiplier: 1.0
          min_depth: 16
          conv_hyperparams {
            regularizer {
              l2_regularizer {
                weight: 3.99999989895e-05
              }
            }
            initializer {
              random_normal_initializer {
                mean: 0.0
                stddev: 0.00999999977648
              }
            }
            activation: RELU_6
            batch_norm {
              decay: 0.97000002861
              center: true
              scale: true
              epsilon: 0.0010000000475
            }
          }
          override_base_feature_extractor_hyperparams: true
        }
        box_coder {
          faster_rcnn_box_coder {
            y_scale: 10.0
            x_scale: 10.0
            height_scale: 5.0
            width_scale: 5.0
          }
        }
        matcher {
          argmax_matcher {
            matched_threshold: 0.5
            unmatched_threshold: 0.5
            ignore_thresholds: false
            negatives_lower_than_unmatched: true
            force_match_for_each_row: true
            use_matmul_gather: true
          }
        }
        similarity_calculator {
          iou_similarity {
          }
        }
        box_predictor {
          convolutional_box_predictor {
            conv_hyperparams {
              regularizer {
                l2_regularizer {
                  weight: 3.99999989895e-05
                }
              }
              initializer {
                random_normal_initializer {
                  mean: 0.0
                  stddev: 0.00999999977648
                }
              }
              activation: RELU_6
              batch_norm {
                decay: 0.97000002861
                center: true
                scale: true
                epsilon: 0.0010000000475
              }
            }
            min_depth: 0
            max_depth: 0
            num_layers_before_predictor: 0
            use_dropout: false
            dropout_keep_probability: 0.800000011921
            kernel_size: 1
            box_code_size: 4
            apply_sigmoid_to_scores: false
            class_prediction_bias_init: -4.59999990463
          }
        }
        anchor_generator {
          ssd_anchor_generator {
            num_layers: 6
            min_scale: 0.20000000298
            max_scale: 0.949999988079
            aspect_ratios: 1.0
            aspect_ratios: 2.0
            aspect_ratios: 0.5
            aspect_ratios: 3.0
            aspect_ratios: 0.333299994469
          }
        }
        post_processing {
          batch_non_max_suppression {
            score_threshold: 0.300000011921
            iou_threshold: 0.600000023842
            max_detections_per_class: 100
            max_total_detections: 100
          }
          score_converter: SIGMOID
        }
        normalize_loss_by_num_matches: true
        loss {
          localization_loss {
            weighted_smooth_l1 {
            }
          }
          hard_example_miner {
            num_hard_examples: 3000
            iou_threshold: 0.99
            loss_type: CLASSIFICATION
            max_negatives_per_positive: 6
            min_negatives_per_image: 0
          }
          classification_loss {
           weighted_sigmoid {

            }
          }
          classification_weight: 1.0
          localization_weight: 1.0
        }
        encode_background_as_zeros: true
        normalize_loc_loss_by_codesize: true
        inplace_batchnorm_update: true
        freeze_batchnorm: false
      }
    }
    train_config {
      batch_size: 96
      data_augmentation_options {
        random_horizontal_flip {
        }
      }
      data_augmentation_options {
        ssd_random_crop {
        }
      }
      sync_replicas: true
      optimizer {
        adam_optimizer: {
          learning_rate: {
            manual_step_learning_rate {
              initial_learning_rate: .0002
              schedule {
                step: 500
                learning_rate: .00003
              }
              schedule {
                step: 1000
                learning_rate: .000003
              }
              schedule {
                step: 3000
                learning_rate: .0000003
              }
              schedule {
                step: 6000
                learning_rate: .00000003
              }
            }
          }
        }
        use_moving_average: false
      }
      fine_tune_checkpoint: "ssd_mobilenet_v2_coco_2018_03_29/model.ckpt"
      from_detection_checkpoint: true
      load_all_detection_checkpoint_vars: true
      num_steps: 50000
      startup_delay_steps: 0.0
      replicas_to_aggregate: 8
      max_number_of_boxes: 100
      unpad_groundtruth_tensors: false
      freeze_variables:
            [ 'FeatureExtractor/MobilenetV2/Conv/',
              'FeatureExtractor/MobilenetV2/expanded_conv/',
              'FeatureExtractor/MobilenetV2/expanded_conv_1/',
              'FeatureExtractor/MobilenetV2/expanded_conv_2/',
              'FeatureExtractor/MobilenetV2/expanded_conv_3/',
              'FeatureExtractor/MobilenetV2/expanded_conv_4/',
              'FeatureExtractor/MobilenetV2/expanded_conv_5/',
              'FeatureExtractor/MobilenetV2/expanded_conv_6/',
              'FeatureExtractor/MobilenetV2/expanded_conv_7/']
    }
    train_input_reader {
      label_map_path: "data/wildlife_cct2_tfrecord/animals_label_map.pbtxt"
      tf_record_input_reader {
        input_path: "data/wildlife_cct2_tfrecord/animals_train.tfrecord"
      }
    }
    eval_config: {
      num_examples: 20
      num_visualizations: 20
      eval_interval_secs: 0
    }
    eval_input_reader {
      label_map_path: "data/wildlife_cct2_tfrecord/animals_label_map.pbtxt"
      shuffle: false
      num_readers: 1
      tf_record_input_reader {
        input_path: "data/wildlife_cct2_tfrecord/animals_val.tfrecord"
      }
    }
    graph_rewriter {
      quantization {
        delay: 48000
        weight_bits: 8
        activation_bits: 8
      }
    }
    ''')

In [ ]:
!pip3 install gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1WOUgE1L5q8ovDvyIa7chksSrE9W2ZBaK

In [ ]:
!unzip wildlife_cct2_tfrecord.zip 'wildlife_cct2_tfrecord/*' -d data

In [ ]:
# !wget https://github.com/HanYangZhao/tf_wildlife_detector/releases/download/1/wildlife.v5.tfrecord.zip
# !unzip wildlife.v5.tfrecord.zip 'wildlife.v5.tfrecord/*' -d data

In [ ]:
# !python legacy/train.py --logtostderr --train_dir=training/ --pipeline_config_path=ssd_mobilenet_v2_coco.config

In [ ]:
!python model_main.py \
  --pipeline_config_path=ssd_mobilenet_v2_coco.config\
  --model_dir=training/ \
  --num_train_steps=55000 \
  --num_eval_steps=2000


**TfLite conversion**

In [ ]:
!python3 export_tflite_ssd_graph.py \
  --pipeline_config_path="ssd_mobilenet_v2_coco.config" \
  --trained_checkpoint_prefix="training/model.ckpt-22758" \
  --output_directory="training" \
  --add_postprocessing_op=true

In [ ]:
!tflite_convert \
  --output_file="training/output_tflite_graph.tflite" \
  --graph_def_file="training/tflite_graph.pb" \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays="normalized_input_image_tensor" \
  --output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -


!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list


!sudo apt-get update

!sudo apt-get install edgetpu-compiler

In [ ]:
!edgetpu_compiler ./training/output_tflite_graph.tflite